In [32]:
import numpy as np, matplotlib.pyplot as plt
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import json
import sys
from scipy import stats
from sklearn.cross_validation import *

%matplotlib inline

In [57]:
matchupdf = pd.read_csv("temp_matchupdf.csv")

In [58]:
matchupdf.head()

,AB,H,PA,RL,SAC,SO,TB,W,bID,matchID,pID,AVG,OBP,ov_AB,ov_H,ov_PA,ov_SAC,ov_SO,ov_TB,ov_W,ov_FACED,ov_AVG,ov_OBP,ov_SO_PCT,ov_W_PCT,ov_H_PCT,ovp_AB,ovp_H,ovp_PA,ovp_SAC,ovp_SO,ovp_TB,ovp_W,ovp_FACED,ovp_SO_PCT,ovp_W_PCT,ovp_H_PCT,ovp_AVG,ovp_OBP
0,5,1,5,R,0,2,1,0,torrc001,torrc001_medlk001,medlk001,0.200000,0.200000,5,1,5,0,2,1,0,1,0.200000,0.200000,0.400000,0.00000,0.200000,1405,357,1502,13,297,674,97,188,0.197736,0.064581,0.237683,0.254093,0.302264
1,3,0,3,L,1,2,0,0,kinnm002,kinnm002_leita001,leita001,0.000000,0.000000,3,0,3,1,2,0,0,1,0.000000,0.000000,0.666667,0.00000,0.000000,5014,1260,5776,91,1023,2335,762,356,0.177112,0.131925,0.218144,0.251296,0.350069
2,6,0,6,R,0,0,0,0,telit001,telit001_felds001,felds001,0.000000,0.000000,12,2,12,0,2,4,0,2,0.166667,0.166667,0.166667,0.00000,0.166667,3602,979,3985,53,556,1835,383,347,0.139523,0.096110,0.245671,0.271793,0.341782
3,6,2,6,R,0,2,4,0,telit001,telit001_grays001,grays001,0.333333,0.333333,12,2,12,0,2,4,0,2,0.166667,0.166667,0.166667,0.00000,0.166667,793,177,864,9,189,289,71,110,0.218750,0.082176,0.204861,0.223203,0.287037
4,3,0,4,R,2,3,0,1,schic002,schic002_adamt001,adamt001,0.000000,0.250000,411,66,424,62,143,84,13,72,0.160584,0.186321,0.337264,0.03066,0.155660,2076,566,2346,34,399,937,270,243,0.170077,0.115090,0.241262,0.272640,0.356351


In [21]:
def compute_supports(df):
    ubids=df.bID.unique()
    pitch = df.groupby('pID').bID.unique()
    bdict={}
    for e,v in zip(pitch.index.values, pitch.values):
        bdict[e] = np.array([item in v for item in ubids])
    pitchers=bdict.keys()
    supports=[]
    for i,p1 in enumerate(pitchers):
        for j,p2 in enumerate(pitchers):
            if  i < j:
                supmask = (bdict[p1] & bdict[p2])
                common_batters = np.sum(supmask)
                supports.append(common_batters)
    print "mean support",np.mean(supports), "median support", np.median(supports)
    return supports, bdict

In [37]:
def recompute_frame(ldf):
    """
    takes a dataframe ldf, makes a copy of it, and returns the copy
    with all conglomerations recomputed
    this is used when a frame is subsetted.
    """
    ldfb=ldf.groupby('bID')
    ldfp=ldf.groupby('pID')
    nldf=ldf.copy()
    #Conglomerate pitcher stats
    nldf.set_index(['pID'], inplace=True)
    for col in ['AB', 'PA', 'H', 'TB', 'SAC', 'SO', 'W']:
        nldf['ovp_'+col] = ldfp[col].sum()
    nldf['ovp_AVG'] = nldf['ovp_H']/nldf['ovp_AB']
    nldf['ovp_FACED']= ldfp.AB.count()
    nldf['ovp_OBP'] = (nldf['ovp_H'] + nldf['ovp_W'])/nldf['ovp_PA']
    for col in ['SO', 'W', 'H']:
        nldf['ovp_' + col + '_PCT'] = nldf['ovp_' + col] / nldf['ovp_PA']
    nldf.reset_index(inplace=True)
    
    #Conglomerate batter stats
    nldf.set_index(['bID'], inplace=True)
    for col in ['AB', 'PA', 'H', 'TB', 'SAC', 'SO', 'W']:
        nldf['ov_'+col] = ldfb[col].sum()
    nldf['ov_AVG'] = nldf['ov_H']/nldf['ov_AB']
    nldf['ov_FACED']= ldfb.AB.count()
    nldf['ov_OBP'] = (nldf['ov_H'] + nldf['ov_W'])/nldf['ov_PA']
    for col in ['SO', 'W', 'H']:
        nldf['ov_' + col + '_PCT'] = nldf['ov_' + col] / nldf['ov_PA']
    nldf.reset_index(inplace=True)
    return nldf

In [23]:
def make_smaller(df, pacountp, pacountb):
    smallidf1=df[(df.ovp_PA > pacountp) & (df.ov_PA > pacountb)]
    smalldf=recompute_frame(smallidf1)
    return smalldf

In [24]:
def make_smaller_2(df, col, limit, greater=True):
    if greater:
        smallidf = df[(df[col] > limit)]
    else:
        smallidf = df[(df[col] < limit)]
    return recompute_frame(smallidf)

Split into test and training sets

In [ ]:
smalldf[smalldf.user_review_count > 12].shape

In [ ]:
trainlist=[]
testlist=[]
validatelist=[]
take=5
for k, v in smalldf.groupby('user_id'):
    if np.mean(v.user_review_count) > 12:
        takenos=np.random.choice(range(take), size=take/2, replace=False)
        takelist=np.array([e in takenos for e in range(take)])
        validatelist.append(v[-take:][~takelist])#use those 
        testlist.append(v[-take:][takelist])#use the other 
        trainlist.append(v[:-take])
    else:
        trainlist.append(v)
traindf=pd.concat(trainlist)
validatedf=pd.concat(validatelist)
testdf=pd.concat(testlist)
print traindf.shape, validatedf.shape, testdf.shape

In [ ]:
maskval=[e in set(validatedf.business_id).difference(set(traindf.business_id)) for e in validatedf.business_id] 
masktest=[e in set(testdf.business_id).difference(set(traindf.business_id)) for e in testdf.business_id] 
print np.sum(maskval), np.sum(masktest)

In [ ]:
traindf=recompute_frame(traindf)
validatedf=recompute_frame(validatedf)
testdf=recompute_frame(testdf)
validatedf=validatedf[['user_id', 'business_id','stars', 'review_id']]
testdf=testdf[['user_id', 'business_id', 'stars', 'review_id']]
traindf.head()

In [ ]:
traindf.to_csv("tempdata/strain.csv", index=False, header=True, encoding="utf-8")
validatedf.to_csv("tempdata/svalidate.csv", index=False, header=True, encoding="utf-8")
testdf.to_csv("tempdata/stest.csv", index=False, header=True, encoding="utf-8")

In [ ]:
ybar = traindf.stars.mean()
ybar

In [ ]:
uuids=traindf.user_id.unique()#unique-user-ids
uiids=traindf.business_id.unique()#unique-item-ids

Compare results function from pset 4

In [25]:
def compare_results(stars_actual, stars_predicted, ylow=1, yhigh=6, model="", predicteds=False, onsame=False, axis=False):
    """
    plot predicted results against actual results. Takes 2 arguments: a
    numpy array of actual ratings and a numpy array of predicted ratings
    scatterplots the predictions, a unit slope line, line segments joining the mean,
    and a filled in area of the standard deviations."
    """
    if onsame:
        ax=onsame
    elif axis:
        ax=axis
    else:
        fig=plt.figure()
        ax=plt.gca()
    df=pd.DataFrame(dict(actual=stars_actual, predicted=stars_predicted))
    xp=[]
    yp=[]
    for k,v in df.groupby('actual'):
        xp.append(k)
        yp.append(v.predicted.mean())        
    cl, = ax.plot(xp,yp, 's-', label="means for %s" % model)
    c=cl.get_color()
    sig=df.groupby('actual').predicted.std().values
    ax.fill_between(xp, yp - sig, yp + sig, color=c, alpha=0.2)
    if predicteds:
        ax.plot(df.actual, df.predicted, '.', color=c, alpha=0.1, label="predicted for %s" % model)

    if not onsame:
        ax.plot([1,5],[1,5], 'k', label="slope 1")
        ax.set_xlabel("actual")
        ax.set_ylabel("predicted")
        ax.set_ylim([ylow,yhigh])
        ax.set_xlim([0.9, 5.1])
    ax.legend(frameon=False, loc="upper left")
    rmse=get_rmse(stars_actual, stars_predicted)
    print "RMSE for %s" % model, rmse
    return ax,rmse

Below is an idea for a function that can be used to determine batter similarity. The function takes the batter-pitcher pair to predict, and it returns similar batters who have faced the given pitcher.

In [17]:
def sim_batters(bID, pID, df):
    batter_row = df.loc[df['bID'] == bID]
    AVG = batter_row['AVG']
    OBP = batter_row['OBP']
    if (batter_row['RL'] == 'R'):
        hand = 0
    else:
        hand = 1
    SAC = batter_row['SAC']
    # continue for other statistics
    
    # find list of bIDs who have faced given pID
    p_batters = df.loc[df['pID'] == pID]
    
    similarity = {}
    # for each batter, calculate similarity
    for b in p_batters:
        comp_b_row = df.loc[df['bID'] == b]
        comp_AVG = batter_row['AVG']
        comp_OBP = batter_row['OBP']
        if (batter_row['RL'] == 'R'):
            comp_hand = 0
        else:
            comp_hand = 1
        comp_SAC = batter_row['SAC']
        # continue for other statistics
        
        #calculate similarity
        AVG_score = AVG_weight*(AVG - comp_AVG)/AVG
        OBP_score = OBP_weight*(OBP - comp_OPB)/OBP
        if (hand == comp_hand):
            hand_score = hand_weight
        else:
             hand_score = 0   
        SAC_score = SAC_weight*(SAC - comp_SAC)/SAC
        total_score = AVG_score + OBP_score + hand_score + SAC_score
        similarity[b] = total_score
    # return top x bIDs with lowest scores
    sorted_sim = sorted(similarity.items(), key=operator.itemgetter(1))
    sim_batters = sorted_sim.keys
    return sim_batters[:10]
    

Below are functions from pset 4 that might be useful for the KNN model

In [40]:
from scipy.stats.stats import pearsonr
# calculate pearson sim between 2 pitchers
def pearson_sim(pitcher1, pitcher2, n_common):
    p1_avg = pitcher1['ov_AVG']
    p2_avg = pitcher2['ov_AVG']
    p1_actual = pitcher1['AVG']
    p2_actual = pitcher2['AVG']
    norm1 = []
    norm2 = []
    for i in range(len(rest1)):
        norm1.append(rest1[i] - p1_actual[i])
        norm2.append(rest2[i] - p2_actual[i])
    if (n_common == 0) or (n_common==1):
        rho = 0;
    else:
        rho = sp.stats.pearsonr(norm1,norm2)[0]
    if np.isnan(rho):
        return 0;
    return rho

In [41]:
def get_restaurant_reviews(pID, df, set_of_batters):
    """
    given a pitcher id and a set of batters, return the sub-dataframe of their
    averages.
    """
    mask = (df.user_id.isin(set_of_batters)) & (df.pID==pID)
    avgs = df[mask]
    avgs = avgs[avgs.user_id.duplicated()==False]
    return avgs

In [42]:
class Database:
    "A class representing a database of similarities and common supports"
    
    def __init__(self, rindexmap, supports):
        "the constructor, takes a map of restaurant id's to integers"
        database={}
        self.rindexmap=rindexmap
        self.supports=supports
        l_keys=len(self.rindexmap.keys())
        self.database_sim=np.zeros([l_keys,l_keys])
        self.database_sup=np.zeros([l_keys, l_keys], dtype=np.int)

    def set_supports(self, supports):
        self.supports=supports
        
    def get(self, b1, b2):
        "returns a tuple of similarity,common_support given two business ids"
        sim=self.database_sim[self.rindexmap[b1]][self.rindexmap[b2]]
        nsup=self.database_sup[self.rindexmap[b1]][self.rindexmap[b2]]
        return (sim, nsup)

In [43]:
db=Database(uiidmap, supports)

NameError: name 'uiidmap' is not defined

In [44]:
def mapper1(row):
    return row[1], (row[2], row[5], row[14])

In [45]:
def combiner(items):
    indict={}
    for key, value in items:
        if not indict.has_key(key):
            indict[key]=[]
        indict[key].append(value)
    return indict.items()

In [46]:
def reducer1(the_input):
    bID, values = the_input
    avgs=[]
    for pID,AVG,ov_AVG in values:
        avgs.append((pID,(AVG, ov_AVG)))
    return bID, avgs

In [47]:
from itertools import combinations_with_replacement
def mapper2(list_input):
    nlist = []
    comb = list(combinations_with_replacement(list_input[1], 2))
    for item in comb:
        if item[0][0] > item[1][0]:
            biz_pair = item[1][0], item[0][0]
            star_pair = item[1][1], item[0][1]
        else:
            biz_pair = item[0][0], item[1][0]
            star_pair = item[0][1], item[1][1]
        tup = (biz_pair, star_pair)
        nlist.append(tup)
    return nlist

In [48]:
def combiner_list(itemslist):
    indict={}
    for items in itemslist:
        for key, value in items:
            if not indict.has_key(key):
                indict[key]=[]
            indict[key].append(value)
    return indict.items()

In [49]:
def reducer2(item_dict):
    p1_id = item_dict[0][0]
    p2_id = item_dict[0][1]
    AVG_1 = [x[0][0] for x in item_dict[1]]
    AVG_2 = [x[1][0] for x in item_dict[1]]
    ov_AVG_1 = [x[0][1] for x in item_dict[1]]
    ov_AVG_2 = [x[1][1] for x in item_dict[1]]
    n_common = len(item_dict[1])
    p1_dict = {'ov_AVG': ov_AVG_1, 'AVG': AVG_1}
    p2_dict = {'ov_AVG': ov_AVG_2, 'AVG': AVG_2}
    rho = pearson_sim(p1_dict, p2_dict, n_common)
    return (p1_id, p2_id),(rho, n_common)

In [50]:
def map_reduce(tuples):
    mapped1=map(mapper1, tuples)
    combine1=combiner(mapped1)
    reduced1=reduce(lambda x,y: x + [reducer1(y)], combine1, [])
    mapped2=map(mapper2,reduced1)
    combine2=combiner_list(mapped2)
    output=reduce(lambda x,y: x + [reducer2(y)], combine2, [])
    return output

In [51]:
tuples=traindf.itertuples()
sims=map_reduce(tuples)

KeyboardInterrupt: 

In [52]:
def populate_from_mr(db, df, mapredlist):
    for tpair,vpair in mapredlist:
        i1=db.rindexmap[tpair[0]]
        i2=db.rindexmap[tpair[1]]
        db.database_sim[i1][i2]=vpair[0]
        db.database_sup[i1][i2]=vpair[1]
        db.database_sim[i2][i1]=vpair[0]
        db.database_sup[i2][i1]=vpair[1]

In [ ]:
populate_from_mr(db, traindf, sims)

Checking work with populate_by_calculating

In [ ]:
db2=Database( uiidmap, supports)

In [53]:
def calculate_similarity(db, df, p1, p2, similarity_func):
    # find common reviewers
    common_reviewers = db.supports[db.rindexmap[p1]][db.rindexmap[p2]]
    n_common=len(common_reviewers)
    if p1==p2:
        return 1., n_common
    #get reviews
    p1_ov_AVG = get_restaurant_reviews(p1, df, common_reviewers)
    p2_ov_AVG = get_restaurant_reviews(p2, df, common_reviewers)
    sim=similarity_func(p1_ov_AVG, p2_ov_AVG, n_common)
    return sim, n_common

def populate_by_calculating(db, df, similarity_func):
    """
    a populator for every pair of businesses in df. takes similarity_func like
    pearson_sim as argument
    """
    items=db.rindexmap.items()
    for b1, i1 in items:
        for b2, i2 in items:
            if i1 <= i2:
                sim, nsup=calculate_similarity(db, df, b1, b2, similarity_func)
                db.database_sim[i1][i2]=sim
                db.database_sim[i2][i1]=sim
                db.database_sup[i1][i2]=nsup
                db.database_sup[i2][i1]=nsup

In [ ]:
%%time
populate_by_calculating(db2, traindf, pearson_sim)

In [ ]:
tpair=('FV0BkoGOd3Yu_eJnXY15ZA', 'O-Xa9GCFWI65YiBD5Jw_hA')
print db.get(tpair[0],tpair[1]),db2.get(tpair[0],tpair[1])

In [54]:
def shrunk_sim(sim, n_common, reg=3.):
    "takes a similarity and shrinks it down by using the regularizer"
    ssim=(n_common*sim)/(n_common+reg)
    return ssim

In [ ]:
"""
Function
--------
knearest

Parameters
----------
restaurant_id : string
    The id of the restaurant whose nearest neighbors we want
set_of_restaurants : array
    The set of restaurants from which we want to find the nearest neighbors
dbase : instance of Database class.
    A database of similarities, on which the get method can be used to get the similarity
  of two businesses. e.g. dbase.get(rid1,rid2)
k : int
    the number of nearest neighbors desired, default 7
reg: float
    the regularization.
    
  
Returns
--------
A sorted list
    of the top k similar restaurants. The list is a list of tuples
    (business_id, shrunken similarity, common support).
"""
from operator import itemgetter
def knearest(restaurant_id, set_of_restaurants, dbase, k=7, reg=3.):
    """
    Given a restaurant_id, dataframe, and database, get a sorted list of the
    k most similar restaurants from the set of restaurants.
    """
    similars=[]
    for other_rest_id in set_of_restaurants:
        if other_rest_id!=restaurant_id:
            sim, nc=dbase.get(restaurant_id, other_rest_id)
            ssim=shrunk_sim(sim, nc, reg=reg)
            simdist=(1. - ssim)/2.
            similars.append((other_rest_id, simdist, nc ))
    similars=sorted(similars, key=itemgetter(1))
    return similars[0:k]

In [ ]:
def get_users_restaurants(df, user_id):
    dfuser=df[df.user_id==user_id]
    dfuserdedup=dfuser.drop_duplicates('business_id')
    return dict(zip(dfuserdedup.business_id.values, dfuserdedup.stars.values))

In [ ]:
"""
Function
--------
rating

Parameters
----------
set_of_restaurants: Dictionary
    The dictionary of restaurant: star-rating pairs you want to make the prediction from.
    This would be the output of a function like get_users_restaurants
train_map: Dictionary
    A dictionary with keys mean, users and items which have estimates of
    overall average or intercept, user coefficients(averages), and
    item coefficients(averages) respectively
dbase : instance of Database class.
    A database of similarities, on which the get method can be used to get the similarity
  of two businessed. e.g. dbase.get(rid1,rid2)
restaurant_id : string
    The id of the restaurant whose nearest neighbors we want
user_id : string
    The id of the user, in whose reviewed restaurants we want to find the neighbors
k : int
    the number of nearest neighbors desired, default 7
reg: float
    the regularization.
    
  
Returns
--------
A float
    which is the imputed rating that we predict that user_id will make for restaurant_id
    
Notes
--------
If the sum of scores is 0, return the baseline estimate of the ranking.
"""
#your code here
# Note: this function was inspired in part by the solutions to the 2013 hw4
def rating(set_of_restaurants, train_map, dbase, restaurant_id, user_id, k=7, reg=3.):
    mu=train_map['mean']
    user_bias = train_map['users'][user_id]
    nsum=0.
    scoresum=0.
    nears=knearest(restaurant_id, set_of_restaurants, dbase, k=k, reg=reg)
    restaurant_bias=train_map['items'][restaurant_id]
    scores=[]
    for r,s,nc in nears:
        ssim = 1-s
        scoresum=scoresum+ssim
        scores.append(ssim)
        r_biases = train_map['items'][r]
        r_stars = set_of_restaurants[r]
        rminusb=(r_stars - (r_biases + user_bias + mu))
        nsum=nsum+ssim*rminusb
    baseline=(user_bias +restaurant_bias + mu)
    if scoresum > 0.:
        val =  nsum/scoresum + baseline
    else:
        val=baseline
    return val

In [ ]:
trainuser=traindf.loc[0].user_id
testrest=testdf[testdf.user_id==trainuser].business_id.values[0]
print trainuser, testrest

In [ ]:
def get_actual(df, userid, bizid):
    return df[(df.user_id==userid) & (df.business_id==bizid)]['stars'].values[0]

print "Actual", get_actual(testdf, trainuser, testrest)
print "Predicted",rating(get_users_restaurants(traindf, trainuser), train_avgs, db, testrest, trainuser, k=2, reg=3.)

In [ ]:
def get_ratings_user_nbd(indf, traindf, train_map, db, k=2, reg=3.):
    zips=zip(indf.business_id, indf.user_id, indf.stars)
    preds=[]
    actuals=[]
    for (r,u,actual) in zips:
        pred=rating(get_users_restaurants(traindf, u),train_map, db, r,u, k, reg)
        preds.append(pred)
        actuals.append(actual)
    return np.array(preds), np.array(actuals)

In [ ]:
pt, at = get_ratings_user_nbd(traindf, traindf, train_avgs, db, k=4, reg=4.)
compare_results(at,pt, model="knn(user) on training k=4, reg=4", predicteds=True)